In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd

In [5]:
dataset = pd.read_excel('/content/drive/MyDrive/code/تحلیل احساسات/Train.xlsx')
dataset.head()

,text,plo
0,من میخواستم برم ببینم ولی یکی به خاطر حضور رضا...,0
1,افتضاح ترین فیلمی که دیدم,0
2,با خانواده رفتیم اصلا خوب نبود,0
3,نمیدانم چی فکر می کنند که این جور فیلم های بی ...,0
4,سلام به همگی لطفا سرمایه ارزشمندتونو از جمله و...,0


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(dataset['text'])


In [7]:
from sklearn.model_selection import cross_val_score

In [8]:
from sklearn import svm
from sklearn.metrics import classification_report
classifier_linear = svm.SVC(kernel='linear')
scores = cross_val_score(classifier_linear,train_vectors, dataset['plo'], cv=5)
scores.mean()


0.9283333333333333

In [9]:
from sklearn.ensemble import BaggingClassifier
model_1 = BaggingClassifier(base_estimator=svm.SVC(kernel='linear'), n_estimators=10, random_state=10)
scores = cross_val_score(model_1,train_vectors, dataset['plo'], cv=10)
scores.mean()

0.9337499999999999

In [10]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import GlobalMaxPooling1D,Flatten,Dropout,Dense, GRU, Embedding, LSTM,Conv1D,BatchNormalization
from tensorflow.keras.layers import Dropout,GlobalMaxPooling1D,Conv1D,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [11]:
stop_words =['به','شکل','ممکن','میشه','دلیل','رو','فقط','در','خانم','آقای','اگر',
            'این','هم','اسامی','و','حال','های',
            'ها','یه','ای','بابت','از','تا','هفتاد'
            ,'درصد''دو','اول','سیمرغ','زن','تاریخ','هاش','اش'
            ,'معادی'
            ,'دوم','ام','۱۰','نفر','تیکت','لباس','خدا'
             ,'سحر','۲۰','۴','کامنت','ش','هیس','شش'
            ]
#پیش پردازش
import re
def preprocess_text(sentence):
    # Removing multiple spaces
    sentence = re.sub(r'@\S+', '', sentence)
    sentence = re.sub(r'!\S+', '', sentence)
    sentence = re.sub(r'؟\S+', '', sentence)
    sentence = re.sub(r'[.]', ' ', sentence)
    sentence = re.sub(r'[/]', '', sentence)
    sentence = re.sub(r'[،]', ' ', sentence)
    sentence = re.sub(r'[؛]', '', sentence)
    sentence = sentence.split()
    sentence =[word for word in sentence if word not in stop_words]
    sentence = ' '.join(sentence)
    return sentence
def ReplacetwoMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
    return pattern.sub(r"\1", s)
dataset['clean_text'] = dataset['text'].apply(ReplacetwoMore)

X = []
sentences = list(dataset["clean_text"])
for sen in sentences:
    X.append(preprocess_text(sen))    

In [12]:
target = dataset['plo']
data = dataset['clean_text']
x_train = data
y_train= target
print(x_train.shape)

(2400,)


In [13]:
import nltk
import numpy as np
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import FreqDist
all_words=' '.join(data)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
print ('number unique word:',num_unique_word)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
number unique word: 6441


In [14]:
max_tokens = 58

In [15]:
print(dataset.groupby(["plo"]).size())

plo
0    1200
1    1200
dtype: int64


In [16]:

num_words = num_unique_word
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)

x_train_tokens = tokenizer.texts_to_sequences(x_train)

print(x_train[100])
print(x_train_tokens[100])
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,padding='post')


idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text
print('train shape',x_train_pad.shape)



با سلام واقعا افتضاح بود خیلی خیلی بد بود تو رو خدا پول و وقت تون رو توی چاه نریزید. هیچ دیدگاه و موضوع خاصی پیگیری نمیشه. هیچی نداره
[12, 167, 9, 84, 3, 8, 8, 49, 3, 45, 10, 695, 105, 2, 51, 1530, 10, 141, 2786, 1531, 60, 1085, 2, 55, 288, 2787, 231, 432, 85]
train shape (2400, 58)


In [17]:
from sklearn.model_selection import KFold
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)

In [84]:
from scipy import stats as s


In [29]:
from statistics import mode

In [82]:
kfold = KFold(n_splits=5, shuffle=True)
acc_per_fold_GRU = []
per_per_fold_GRU = []
rec_per_fold_GRU = []
f1_per_fold_GRU = []

acc_per_fold_c = []
per_per_fold_c = []
rec_per_fold_c = []
f1_per_fold_c = []

acc_per_fold_L= []
per_per_fold_L = []
rec_per_fold_L = []
f1_per_fold_L = []


acc_per_fold_L2= []
per_per_fold_L2 = []
rec_per_fold_L2 = []
f1_per_fold_L2 = []

acc_per_fold_av= []
per_per_fold_av = []
rec_per_fold_av = []
f1_per_fold_av = []
fold_no = 1
for train, test in kfold.split(x_train_pad, y_train):   
    embedding_size = 100
    model4 = Sequential()
    model4.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model4.add(Dropout(0.5))
    model4.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model4.add(BatchNormalization())
    model4.add(GlobalMaxPooling1D())
    model4.add(Dense(200, activation='relu'))
    model4.add(Dropout(0.5))
    model4.add(Dense(1,activation='sigmoid'))
    model4.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    ####################################
    embedding_size2 = 150
    model2 = Sequential()
    model2.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size2,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model2.add(Dropout(0.5))
    model2.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model2.add(BatchNormalization())
    model2.add(GlobalMaxPooling1D())
    model2.add(Dense(200, activation='relu'))
    model2.add(Dropout(0.5))
    model2.add(Dense(1,activation='sigmoid'))
    model2.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
  #############
    embedding_size3 = 200
    model5 = Sequential()
    model5.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size3,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model5.add(Dropout(0.5))
    model5.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model5.add(BatchNormalization())
    model5.add(GlobalMaxPooling1D())
    model5.add(Dense(200, activation='relu'))
    model5.add(Dropout(0.5))
    model5.add(Dense(1,activation='sigmoid'))
    model5.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    #############
    embedding_size4 = 250
    model6 = Sequential()
    model6.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size4,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model6.add(Dropout(0.5))
    model6.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model6.add(BatchNormalization())
    model6.add(GlobalMaxPooling1D())
    model6.add(Dense(200, activation='relu'))
    model6.add(Dropout(0.5))
    model6.add(Dense(1,activation='sigmoid'))
    model6.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])




    print('------------------------------------------------------------------------')
    print(f'CNN: Training for fold {fold_no} ...')
    print(" CNN WE 100")
    history = model4.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model4.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred=model4.predict(x_train_pad[test])
    y_pred = np.round(y_pred).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred)
    recall = recall_score(y_train[test], y_pred , average="binary")
    precision = precision_score(y_train[test], y_pred , average="binary")
    f1 = f1_score(y_train[test], y_pred, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'CNN 100: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_GRU.append(accuracy * 100)
    per_per_fold_GRU.append(recall * 100) 
    rec_per_fold_GRU.append(precision * 100) 
    f1_per_fold_GRU.append(f1 * 100) 







    print(" CNN WE 150")
    history2 = model2.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model2.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred2=model2.predict(x_train_pad[test])
    y_pred2 = np.round(y_pred2).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred2)
    recall = recall_score(y_train[test], y_pred2 , average="binary")
    precision = precision_score(y_train[test], y_pred2 , average="binary")
    f1 = f1_score(y_train[test], y_pred2, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'CNN 50: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_c.append(accuracy * 100)
    per_per_fold_c.append(recall * 100) 
    rec_per_fold_c.append(precision * 100) 
    f1_per_fold_c.append(f1 * 100) 
    



    print(" CNN WE 200")
    history3 = model5.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model5.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred3=model5.predict(x_train_pad[test])
    y_pred3 = np.round(y_pred3).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred3)
    recall = recall_score(y_train[test], y_pred3 , average="binary")
    precision = precision_score(y_train[test], y_pred3 , average="binary")
    f1 = f1_score(y_train[test], y_pred3, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'LSTM 50: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_L.append(accuracy * 100)
    per_per_fold_L.append(recall * 100) 
    rec_per_fold_L.append(precision * 100) 
    f1_per_fold_L.append(f1 * 100)



    print(" CNN WE 250")
    history4 = model6.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model6.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred4=model6.predict(x_train_pad[test])
    y_pred4 = np.round(y_pred4).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred4)
    recall = recall_score(y_train[test], y_pred4 , average="binary")
    precision = precision_score(y_train[test], y_pred4 , average="binary")
    f1 = f1_score(y_train[test], y_pred4, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'LSTM 50: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_L2.append(accuracy * 100)
    per_per_fold_L2.append(recall * 100) 
    rec_per_fold_L2.append(precision * 100) 
    f1_per_fold_L2.append(f1 * 100)  




    p1=model4.predict(x_train_pad[test])
    p1 = np.round(p1).astype(int)
    p2=model2.predict(x_train_pad[test])
    p2 = np.round(p2).astype(int)
    p3=model5.predict(x_train_pad[test])
    p3 = np.round(p3).astype(int)
    p4=model6.predict(x_train_pad[test])
    p4 = np.round(p4).astype(int)

    final_pred = np.array([])
    for i in range(0,len(x_train_pad[test])):
      final_pred = np.append(final_pred, s.mode([p1[i][0], p2[i][0], p3[i][0], p4[i][0]])[0]  )
    accuracy=accuracy_score(y_train[test], final_pred)
    recall = recall_score(y_train[test], final_pred , average="binary")
    precision = precision_score(y_train[test], final_pred , average="binary")
    f1 = f1_score(y_train[test], final_pred, average="binary")
    print("Max Voting")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))

    acc_per_fold_av.append(accuracy * 100)
    per_per_fold_av.append(recall * 100) 
    rec_per_fold_av.append(precision * 100) 
    f1_per_fold_av.append(f1 * 100) 


    fold_no = fold_no + 1

------------------------------------------------------------------------
CNN: Training for fold 1 ...
 CNN WE 100
Epoch 1/10
8/8 [==============================] - 1s 60ms/step - loss: 1.0259 - accuracy: 0.5208 - val_loss: 0.6883 - val_accuracy: 0.5000
Epoch 2/10
8/8 [==============================] - 0s 26ms/step - loss: 0.7547 - accuracy: 0.5849 - val_loss: 0.6852 - val_accuracy: 0.6958
Epoch 3/10
8/8 [==============================] - 0s 25ms/step - loss: 0.5175 - accuracy: 0.7458 - val_loss: 0.6803 - val_accuracy: 0.8125
Epoch 4/10
8/8 [==============================] - 0s 23ms/step - loss: 0.4015 - accuracy: 0.8443 - val_loss: 0.6694 - val_accuracy: 0.8562
Epoch 5/10
8/8 [==============================] - 0s 22ms/step - loss: 0.2996 - accuracy: 0.8870 - val_loss: 0.6532 - val_accuracy: 0.8604
Epoch 6/10
8/8 [==============================] - 0s 22ms/step - loss: 0.2276 - accuracy: 0.9198 - val_loss: 0.6369 - val_accuracy: 0.8854
Epoch 7/10
8/8 [==============================] - 0s

In [85]:
print('------------------------------------------------------------------------')
print('CNN 100: Score per fold')
for i in range(0, len(acc_per_fold_GRU)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_GRU[i]}% - Recall: {rec_per_fold_GRU[i]}% - Precision: {per_per_fold_GRU[i]}% - F1: {f1_per_fold_GRU[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 100: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_GRU)} (+- {np.std(acc_per_fold_GRU)})')
print(f'> Recall: {np.mean(rec_per_fold_GRU)} (+- {np.std(rec_per_fold_GRU)})')
print(f'> Precision: {np.mean(per_per_fold_GRU)} (+- {np.std(per_per_fold_GRU)})')
print(f'> F1: {np.mean(f1_per_fold_GRU)} (+- {np.std(f1_per_fold_GRU)})')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('CNN 150: Score per fold')
for i in range(0, len(acc_per_fold_c)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_c[i]}% - Recall: {rec_per_fold_c[i]}% - Precision: {per_per_fold_c[i]}% - F1: {f1_per_fold_c[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 150: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_c)} (+- {np.std(acc_per_fold_c)})')
print(f'> Recall: {np.mean(rec_per_fold_c)} (+- {np.std(rec_per_fold_c)})')
print(f'> Precision: {np.mean(per_per_fold_c)} (+- {np.std(per_per_fold_c)})')
print(f'> F1: {np.mean(f1_per_fold_c)} (+- {np.std(f1_per_fold_c)})')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('CNN 200: Score per fold')
for i in range(0, len(acc_per_fold_L)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_L[i]}% - Recall: {rec_per_fold_L[i]}% - Precision: {per_per_fold_L[i]}% - F1: {f1_per_fold_L[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 200: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_L)} (+- {np.std(acc_per_fold_L)})')
print(f'> Recall: {np.mean(rec_per_fold_L)} (+- {np.std(rec_per_fold_L)})')
print(f'> Precision: {np.mean(per_per_fold_L)} (+- {np.std(per_per_fold_L)})')
print(f'> F1: {np.mean(f1_per_fold_L)} (+- {np.std(f1_per_fold_L)})')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('CNN 250: Score per fold')
for i in range(0, len(acc_per_fold_L2)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_L2[i]}% - Recall: {rec_per_fold_L2[i]}% - Precision: {per_per_fold_L2[i]}% - F1: {f1_per_fold_L2[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 250: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_L2)} (+- {np.std(acc_per_fold_L2)})')
print(f'> Recall: {np.mean(rec_per_fold_L2)} (+- {np.std(rec_per_fold_L2)})')
print(f'> Precision: {np.mean(per_per_fold_L2)} (+- {np.std(per_per_fold_L2)})')
print(f'> F1: {np.mean(f1_per_fold_L2)} (+- {np.std(f1_per_fold_L2)})')
print('------------------------------------------------------------------------')



print('-------------------------------Max Voting -------------------------------')
print('Max Voting: Score per fold')
for i in range(0, len(acc_per_fold_av)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_av[i]}% - Recall: {rec_per_fold_av[i]}% - Precision: {per_per_fold_av[i]}% - F1: {f1_per_fold_av[i]}%')
  
print('------------------------------------------------------------------------')
print('Max Voting: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_av)} (+- {np.std(acc_per_fold_av)})')
print(f'> Recall: {np.mean(rec_per_fold_av)} (+- {np.std(rec_per_fold_av)})')
print(f'> Precision: {np.mean(per_per_fold_av)} (+- {np.std(per_per_fold_av)})')
print(f'> F1: {np.mean(f1_per_fold_av)} (+- {np.std(f1_per_fold_av)})')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
CNN 100: Score per fold
------------------------------------------------------------------------
> Fold 1  - Accuracy: 91.875% - Recall: 88.50574712643679% - Precision: 96.25% - F1: 92.21556886227546%
------------------------------------------------------------------------
> Fold 2  - Accuracy: 92.70833333333334% - Recall: 94.61883408071749% - Precision: 90.17094017094017% - F1: 92.34135667396062%
------------------------------------------------------------------------
> Fold 3  - Accuracy: 91.66666666666666% - Recall: 88.01498127340824% - Precision: 96.70781893004116% - F1: 92.15686274509804%
------------------------------------------------------------------------
> Fold 4  - Accuracy: 89.375% - Recall: 85.44776119402985% - Precision: 95.0207468879668% - F1: 89.9803536345776%
------------------------------------------------------------------------
> Fold 5  - Accuracy: 94.79166666666666% - Recall: 93.927125506072

In [86]:
kfold = KFold(n_splits=10, shuffle=True)
acc_per_fold_GRU = []
per_per_fold_GRU = []
rec_per_fold_GRU = []
f1_per_fold_GRU = []

acc_per_fold_c = []
per_per_fold_c = []
rec_per_fold_c = []
f1_per_fold_c = []

acc_per_fold_L= []
per_per_fold_L = []
rec_per_fold_L = []
f1_per_fold_L = []


acc_per_fold_L2= []
per_per_fold_L2 = []
rec_per_fold_L2 = []
f1_per_fold_L2 = []

acc_per_fold_av= []
per_per_fold_av = []
rec_per_fold_av = []
f1_per_fold_av = []
fold_no = 1
for train, test in kfold.split(x_train_pad, y_train):   
    embedding_size = 100
    model4 = Sequential()
    model4.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model4.add(Dropout(0.5))
    model4.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model4.add(BatchNormalization())
    model4.add(GlobalMaxPooling1D())
    model4.add(Dense(200, activation='relu'))
    model4.add(Dropout(0.5))
    model4.add(Dense(1,activation='sigmoid'))
    model4.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    ####################################
    embedding_size2 = 150
    model2 = Sequential()
    model2.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size2,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model2.add(Dropout(0.5))
    model2.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model2.add(BatchNormalization())
    model2.add(GlobalMaxPooling1D())
    model2.add(Dense(200, activation='relu'))
    model2.add(Dropout(0.5))
    model2.add(Dense(1,activation='sigmoid'))
    model2.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
  #############
    embedding_size3 = 200
    model5 = Sequential()
    model5.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size3,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model5.add(Dropout(0.5))
    model5.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model5.add(BatchNormalization())
    model5.add(GlobalMaxPooling1D())
    model5.add(Dense(200, activation='relu'))
    model5.add(Dropout(0.5))
    model5.add(Dense(1,activation='sigmoid'))
    model5.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    #############
    embedding_size4 = 250
    model6 = Sequential()
    model6.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size4,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model6.add(Dropout(0.5))
    model6.add(Conv1D(256,kernel_size=3,padding='same',activation='relu',strides=1))
    model6.add(BatchNormalization())
    model6.add(GlobalMaxPooling1D())
    model6.add(Dense(200, activation='relu'))
    model6.add(Dropout(0.5))
    model6.add(Dense(1,activation='sigmoid'))
    model6.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])




    print('------------------------------------------------------------------------')
    print(f'CNN: Training for fold {fold_no} ...')
    print(" CNN WE 100")
    history = model4.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model4.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred=model4.predict(x_train_pad[test])
    y_pred = np.round(y_pred).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred)
    recall = recall_score(y_train[test], y_pred , average="binary")
    precision = precision_score(y_train[test], y_pred , average="binary")
    f1 = f1_score(y_train[test], y_pred, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'CNN 100: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_GRU.append(accuracy * 100)
    per_per_fold_GRU.append(recall * 100) 
    rec_per_fold_GRU.append(precision * 100) 
    f1_per_fold_GRU.append(f1 * 100) 







    print(" CNN WE 150")
    history2 = model2.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model2.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred2=model2.predict(x_train_pad[test])
    y_pred2 = np.round(y_pred2).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred2)
    recall = recall_score(y_train[test], y_pred2 , average="binary")
    precision = precision_score(y_train[test], y_pred2 , average="binary")
    f1 = f1_score(y_train[test], y_pred2, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'CNN 50: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_c.append(accuracy * 100)
    per_per_fold_c.append(recall * 100) 
    rec_per_fold_c.append(precision * 100) 
    f1_per_fold_c.append(f1 * 100) 
    



    print(" CNN WE 200")
    history3 = model5.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model5.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred3=model5.predict(x_train_pad[test])
    y_pred3 = np.round(y_pred3).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred3)
    recall = recall_score(y_train[test], y_pred3 , average="binary")
    precision = precision_score(y_train[test], y_pred3 , average="binary")
    f1 = f1_score(y_train[test], y_pred3, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'LSTM 50: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_L.append(accuracy * 100)
    per_per_fold_L.append(recall * 100) 
    rec_per_fold_L.append(precision * 100) 
    f1_per_fold_L.append(f1 * 100)



    print(" CNN WE 250")
    history4 = model6.fit(x_train_pad[train], y_train[train],
              batch_size=256,
              epochs=10, validation_data=(x_train_pad[test], y_train[test]),
              verbose=1)
    scores = model6.evaluate(x_train_pad[test], y_train[test], verbose=0)
    y_pred4=model6.predict(x_train_pad[test])
    y_pred4 = np.round(y_pred4).astype(int)
    accuracy=accuracy_score(y_train[test], y_pred4)
    recall = recall_score(y_train[test], y_pred4 , average="binary")
    precision = precision_score(y_train[test], y_pred4 , average="binary")
    f1 = f1_score(y_train[test], y_pred4, average="binary")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))
    print(f'LSTM 50: Score for fold {fold_no}: Accuracy of {accuracy*100}; Recall of {recall*100}; Precision of {precision*100}; F1 of {f1*100}')
    acc_per_fold_L2.append(accuracy * 100)
    per_per_fold_L2.append(recall * 100) 
    rec_per_fold_L2.append(precision * 100) 
    f1_per_fold_L2.append(f1 * 100)  




    p1=model4.predict(x_train_pad[test])
    p1 = np.round(p1).astype(int)
    p2=model2.predict(x_train_pad[test])
    p2 = np.round(p2).astype(int)
    p3=model5.predict(x_train_pad[test])
    p3 = np.round(p3).astype(int)
    p4=model6.predict(x_train_pad[test])
    p4 = np.round(p4).astype(int)

    final_pred = np.array([])
    for i in range(0,len(x_train_pad[test])):
      final_pred = np.append(final_pred, s.mode([p1[i][0], p2[i][0], p3[i][0], p4[i][0]])[0]  )
    accuracy=accuracy_score(y_train[test], final_pred)
    recall = recall_score(y_train[test], final_pred , average="binary")
    precision = precision_score(y_train[test], final_pred , average="binary")
    f1 = f1_score(y_train[test], final_pred, average="binary")
    print("Max Voting")
    print("accuracy:","%.2f" %(accuracy*100))
    print("racall:","%.2f" %(recall*100))
    print("precision:","%.2f" %(precision*100))
    print("f1score:","%.2f" %(f1*100))

    acc_per_fold_av.append(accuracy * 100)
    per_per_fold_av.append(recall * 100) 
    rec_per_fold_av.append(precision * 100) 
    f1_per_fold_av.append(f1 * 100) 


    fold_no = fold_no + 1

------------------------------------------------------------------------
CNN: Training for fold 1 ...
 CNN WE 100
Epoch 1/10
9/9 [==============================] - 2s 105ms/step - loss: 0.8337 - accuracy: 0.5333 - val_loss: 0.6910 - val_accuracy: 0.5458
Epoch 2/10
9/9 [==============================] - 0s 22ms/step - loss: 0.6224 - accuracy: 0.6560 - val_loss: 0.6875 - val_accuracy: 0.6208
Epoch 3/10
9/9 [==============================] - 0s 22ms/step - loss: 0.5015 - accuracy: 0.7662 - val_loss: 0.6749 - val_accuracy: 0.6292
Epoch 4/10
9/9 [==============================] - 0s 21ms/step - loss: 0.3644 - accuracy: 0.8431 - val_loss: 0.6523 - val_accuracy: 0.8792
Epoch 5/10
9/9 [==============================] - 0s 21ms/step - loss: 0.2640 - accuracy: 0.8977 - val_loss: 0.6337 - val_accuracy: 0.8875
Epoch 6/10
9/9 [==============================] - 0s 21ms/step - loss: 0.1936 - accuracy: 0.9255 - val_loss: 0.6201 - val_accuracy: 0.9167
Epoch 7/10
9/9 [==============================] - 0

In [87]:
print('------------------------------------------------------------------------')
print('CNN 100: Score per fold')
for i in range(0, len(acc_per_fold_GRU)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_GRU[i]}% - Recall: {rec_per_fold_GRU[i]}% - Precision: {per_per_fold_GRU[i]}% - F1: {f1_per_fold_GRU[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 100: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_GRU)} (+- {np.std(acc_per_fold_GRU)})')
print(f'> Recall: {np.mean(rec_per_fold_GRU)} (+- {np.std(rec_per_fold_GRU)})')
print(f'> Precision: {np.mean(per_per_fold_GRU)} (+- {np.std(per_per_fold_GRU)})')
print(f'> F1: {np.mean(f1_per_fold_GRU)} (+- {np.std(f1_per_fold_GRU)})')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('CNN 150: Score per fold')
for i in range(0, len(acc_per_fold_c)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_c[i]}% - Recall: {rec_per_fold_c[i]}% - Precision: {per_per_fold_c[i]}% - F1: {f1_per_fold_c[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 150: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_c)} (+- {np.std(acc_per_fold_c)})')
print(f'> Recall: {np.mean(rec_per_fold_c)} (+- {np.std(rec_per_fold_c)})')
print(f'> Precision: {np.mean(per_per_fold_c)} (+- {np.std(per_per_fold_c)})')
print(f'> F1: {np.mean(f1_per_fold_c)} (+- {np.std(f1_per_fold_c)})')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('CNN 200: Score per fold')
for i in range(0, len(acc_per_fold_L)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_L[i]}% - Recall: {rec_per_fold_L[i]}% - Precision: {per_per_fold_L[i]}% - F1: {f1_per_fold_L[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 200: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_L)} (+- {np.std(acc_per_fold_L)})')
print(f'> Recall: {np.mean(rec_per_fold_L)} (+- {np.std(rec_per_fold_L)})')
print(f'> Precision: {np.mean(per_per_fold_L)} (+- {np.std(per_per_fold_L)})')
print(f'> F1: {np.mean(f1_per_fold_L)} (+- {np.std(f1_per_fold_L)})')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('CNN 250: Score per fold')
for i in range(0, len(acc_per_fold_L2)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_L2[i]}% - Recall: {rec_per_fold_L2[i]}% - Precision: {per_per_fold_L2[i]}% - F1: {f1_per_fold_L2[i]}%')
  
print('------------------------------------------------------------------------')
print('CNN 250: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_L2)} (+- {np.std(acc_per_fold_L2)})')
print(f'> Recall: {np.mean(rec_per_fold_L2)} (+- {np.std(rec_per_fold_L2)})')
print(f'> Precision: {np.mean(per_per_fold_L2)} (+- {np.std(per_per_fold_L2)})')
print(f'> F1: {np.mean(f1_per_fold_L2)} (+- {np.std(f1_per_fold_L2)})')
print('------------------------------------------------------------------------')



print('-------------------------------Max Voting -------------------------------')
print('Max Voting: Score per fold')
for i in range(0, len(acc_per_fold_av)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1}  - Accuracy: {acc_per_fold_av[i]}% - Recall: {rec_per_fold_av[i]}% - Precision: {per_per_fold_av[i]}% - F1: {f1_per_fold_av[i]}%')
  
print('------------------------------------------------------------------------')
print('Max Voting: Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_av)} (+- {np.std(acc_per_fold_av)})')
print(f'> Recall: {np.mean(rec_per_fold_av)} (+- {np.std(rec_per_fold_av)})')
print(f'> Precision: {np.mean(per_per_fold_av)} (+- {np.std(per_per_fold_av)})')
print(f'> F1: {np.mean(f1_per_fold_av)} (+- {np.std(f1_per_fold_av)})')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
CNN 100: Score per fold
------------------------------------------------------------------------
> Fold 1  - Accuracy: 92.5% - Recall: 87.2% - Precision: 98.1981981981982% - F1: 92.37288135593221%
------------------------------------------------------------------------
> Fold 2  - Accuracy: 94.16666666666667% - Recall: 93.33333333333333% - Precision: 94.91525423728814% - F1: 94.11764705882352%
------------------------------------------------------------------------
> Fold 3  - Accuracy: 91.66666666666666% - Recall: 93.54838709677419% - Precision: 90.625% - F1: 92.06349206349206%
------------------------------------------------------------------------
> Fold 4  - Accuracy: 93.75% - Recall: 89.91596638655463% - Precision: 97.27272727272728% - F1: 93.44978165938865%
------------------------------------------------------------------------
> Fold 5  - Accuracy: 92.5% - Recall: 91.30434782608695% - Precision: 92.9203539